In [39]:
from bertviz import model_view
from transformers import BertTokenizer, BertForSequenceClassification

model_version = r"E:/PyCharmWorkspace/LegalJudgment/utils/model/transformers_bert/tmp/self"
do_lower_case = True
model = BertForSequenceClassification.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

In [40]:
sentence_a = "被告人周某在本县武康街道营盘小区131幢南侧过道处，采用拉车门的方式在被害人林某浙Ｅ×××××的白色越野车内窃得黑色苹果7PLUS手机一部、白色苹果5手机1部、金色苹果6PLUS手机一部、金色苹果6SPLUS手机1部、外币8张（含10000日元纸币1张、5000韩元纸币1张、10美元纸币1张、5美元纸币2张、50美元纸币2张、10加拿大元纸币1张）以及人民币20元。经鉴定，涉案财物共计价值人民币12159.44元。案发后，部分赃物已被公安机关追回并发还被害人林某。"
# sentence_b = "马某被孙某打伤"
sentence_b = None

hide_delimiter_attn = False

inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']

logits = model(input_ids, token_type_ids=token_type_ids)
attention = logits[-1]

input_id_list = input_ids[0].tolist()  # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

if sentence_b:
    sentence_b_start = token_type_ids[0].tolist().index(1)
else:
    sentence_b_start = None
if hide_delimiter_attn:
    for i, t in enumerate(tokens):
        if t in ("[SEP]", "[CLS]"):
            for layer_attn in attention:
                layer_attn[0, :, i, :] = 0
                layer_attn[0, :, :, i] = 0

# model_view(attention, tokens, sentence_b_start)

In [41]:
import numpy as np
attention_matrix = list(attention)
attention_matrix = [att.detach().numpy() for att in attention_matrix]
attention_matrix = np.sum(attention_matrix, axis=1)
attention_matrix = np.sum(attention_matrix, axis=1)
attention_matrix = np.sum(attention_matrix, axis=0)

In [42]:
for att in attention_matrix:
    print(att)

[16.618574    0.19159304  0.11343668  0.13675603  0.13348415  0.07931524
  0.7846299   0.15115532  0.14451675  0.12259261  0.1155013   0.16949393
  0.16316801  0.1471706   0.15753679  0.28007096  0.2895757   0.5595211
  0.7057139   0.31883594  0.44139498  0.6172066   0.35984352  0.45008564
  1.5951406   1.6083752   1.1204708   1.4198799   1.0828447   0.6786276
  0.5600201   1.7189457   1.3441597   1.9886414   0.36574885  2.5271928
  0.39599967  0.2772905   0.09569227  0.22335188  0.24431093  0.1178331
  0.11582573  0.10136177  0.10952554  0.12024078  0.7141521   0.45196342
  0.63812035  0.32135227  0.458009    2.2785363   2.9656513  11.8917465
  7.694623    0.3385343   0.28514284  0.19296692  0.10552581  0.18558343
  0.45687976  0.17240821  0.20799923  0.30540794  0.14538167  0.32018724
  0.13393363  0.21797888  0.13608168  0.07972562  0.16428667  0.1534582
  0.17126645  0.17931373  0.1594132   0.33068946  0.15225725  0.15719342
  0.1355375   0.07906513  0.23971143  0.26026404  0.14649

In [53]:
words_attention = [(tokens[i],att[i]) for i, att in enumerate(attention_matrix)]
words_attention = words_attention[1:-1]
for i in words_attention:
    print(i)
att = [i[1] for i in words_attention]

('被', 4.6418476)
('告', 9.982815)
('人', 4.6520514)
('周', 5.2032323)
('某', 3.0801604)
('在', 2.8330762)
('本', 3.9856255)
('县', 3.9255157)
('武', 5.256655)
('康', 3.4387167)
('街', 4.5914974)
('道', 4.290794)
('营', 3.3305087)
('盘', 4.193932)
('小', 4.032318)
('区', 2.9211183)
('131', 5.3640203)
('幢', 9.110217)
('南', 4.198291)
('侧', 5.7428474)
('过', 6.1703873)
('道', 5.296853)
('处', 3.3636684)
('，', 6.592551)
('采', 4.801875)
('用', 2.6667738)
('拉', 5.4061975)
('车', 6.072451)
('门', 4.5824404)
('的', 2.8649156)
('方', 3.0601726)
('式', 2.9198174)
('在', 2.087524)
('被', 4.7250953)
('害', 1.950788)
('人', 3.9554281)
('林', 6.2560005)
('某', 3.471056)
('浙', 7.2335987)
('ｅ', 7.4003572)
('##×', 2.8360555)
('##×', 3.4174147)
('##×', 3.4174979)
('##×', 3.0703392)
('##×', 3.9784074)
('的', 3.2253513)
('白', 5.914568)
('色', 5.5122504)
('越', 3.8428488)
('野', 5.304288)
('车', 3.745772)
('内', 6.0369043)
('窃', 6.3799353)
('得', 5.96971)
('黑', 9.504498)
('色', 3.2476296)
('苹', 3.6193066)
('果', 0.73312205)
('7', 2.6586707)
('##

In [56]:
max_attention = np.max(att)
min_attention = np.min(att)
difference = max_attention - min_attention

In [62]:
top_k = 60

limits = max_attention -  0.01 * top_k * difference
_ = [print(i) if i[1] > limits else None for i in words_attention]


('告', 9.982815)
('幢', 9.110217)
('黑', 9.504498)
('鉴', 9.241557)
('##59', 9.638658)
('案', 11.185146)
('并', 20.461737)
